## 3.2 단순한 word2vec
word2vec에서 제안하는 CBOW(continuous bag-of words)모델을 신경망으로 구현해보자.

### 3.2.1. CBOW 모델의 추론 처리
CBOW 모델은 맥락으로부터 타겟을 추측하는 용도의 신경망이다. (타겟은 중앙단어이고 그 주변 단어들이 맥락이다.)
CBOW 모델의 입력은 맥락이다. 맥락은 'you'와 'goodbye'같은 단어들의 목록이다.
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-9.png?raw=true)  
해당 그림이 CBOW모델의 신경망이다. 입력층이 2개 있고, 은닉층을 거쳐 출력층에 도달한다.
- 입력층에서 은닉층으로의 변환은 똑같은 완전연결계층($W_{in}$)이 처리
- 은닉층에서 출력층 뉴런으로의 변환은 다른 완전연결계층($W_{out}$)이 처리
은닉층의 뉴런은 입력층의 완전연결계층에 의해 변환된 값이 되는데, 입력층이 여러개 이면 전체를 '평균'한다.
해당 예제에 따르면 은닉층 뉴런은 $1/2(h_1 + h_2)$가 되는 것이다.  
마지막으로 출력층을을 보면, 출력층의 뉴런은 총 7개인데, 중요한 것은 각 뉴런하나하나가 단어에 대응한다는 것이다. 그리고 출력층 뉴런은 각 단어의 '점수'를 뜻하며, 값이 높을수록 대응단어의 출현 확률도 높아진다. 여기서 점수란 확률로 해석되기 전의 값이고, 해당 점수에 소프트맥스를 적용하면 확률을 얻을 수 있다.
> 해당 그림에서 입력층이 2개인 이유는 맥락으로 고려할 단어를 2개로 정했기 때문. 즉, 맥락에 포함시킬 단어가 N개라면 입력층도 N개가 된다.

![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-11.png?raw=true)  
CBOW 모델의 앞단에는 2개의 Matmul 계층이 있고ㅗ, 이어서 두 계층의 출력이 더해지고 0.5를 곱해주면 평균이 된다. 이 것이 은닉층 뉴런이며 마지막으로 ㅇ느닉층 뉴런에 또 다른 Matmul 계층이 적용되어 점수가 출력된다.

> WARNING_ 편향을 사용하지 않는 완전연결계층의 처리는 Matmul계층의 순전파와 같다. Matmul 계층은 내부에서 행렬 곱을 계산한다.

CBOW를 모델의 추론 처리를 파이썬으로 구현해보자

In [2]:

import numpy as np
from matmul import MatMul

# 샘플 맥락 데이터
c0 = np.array([1, 0, 0, 0, 0, 0, 0])
c1 = np.array([0, 0, 1, 0, 0, 0, 0])

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[ 1.08451151 -0.23632064  0.79267513 -0.54460216  1.39118971  0.98951643
 -1.27282752]


### 3.2.2 CBOW 모델의 학습
지금까지 설명한 CBOW 모델은 출력층에서 각 단어의 점수를 출력했다. 이 점수에 소프트맥스 함수를 적용하면 '확률'을 얻을 수 있다.
이 확률은 맥락(전후 단어)이 주어졌을 때 중앙에 어떤 단어가 출현하는지를 나타낸다.
아래의 예에서 맥락은 'you'와 'goodbye'이고, 정답레이블(신경망이 예측해야 하는 것)은 'say'이다. 

![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-12.png?raw=true)  

![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-13.png?raw=true)  
위 그림처럼 다중 클래스 분류를 수행할 때에는 소프트맥스 함수를 이용해 output(점수)를 확률로 변환하ㅗ, 그 확률과 정답 레이블로부터 교차 엔트로피 값을 손실로 사용해 학습을 진행한다.  Cross Entropy Error 계층을 추가하는 것만으로 손실을 얻을 수 있다.  
이상이 CBOW 모델의 손실을 구하는 계산의 흐름이자, 이 신경망의 순방향 전파이다. 덧붙여 현재는 Softmax계층과 Cross Entropy Error 계층을 사용했지만, 이 계층을 Softmax with Loss 라는 하나의 계층으로 구현할 것이다. 그러면 [그림 3-14]처럼 될 것이다. 

### 3.2.3 word2vec의 가중치와 분산 표현
word2vec에서 사용되는 신경망에는 두 가지 가중치가 있다. 입력 측 가중치와 출력 측 가중치.  
그리고 입력측 가중치의 각 행이 각 단어의 분산 표현에 해당된다. 또한 출력 측 가중치는 수직방향으로 저장되었지만 단어의 의미가 인코딩된 벡터가 저장되고 있다고 생각할 수 있다.
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%203-15.png?raw=true)
그러면 최종적으로 이용하는 단어의 분산으로는 어느 쪽 가중치를 선택하면 좋은가?
- A: 입력 측 가중치만 이용
- B: 출력 측 가중치만 이용
- C : 양쪽 가중치를 모두 이용

word2vec(특히 skip-gram)에서는 A안인 입력 측의 가중치만 이용하는 것이 가장 대중적이다. 많은 연구에서 출력 측 가중치는 버리고 입력 측 가중치만을 최종 단어의 분산표현으로 이용한다. 우리도 이를 따를 것이다.
> NOTE_ [문헌[38]"Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling"](https://arxiv.org/abs/1611.01462)에서는 word2vec의 skip-gram 모델을 대상으로 $W_{in}$의 효과를 실험으로 보여준다. 한편 word2vec과 비슷한 기법인 GloVe에서는 두 가중치를 더했을 때 좋은 결과를 얻었다.